In [30]:
import findspark
findspark.init('/opt/spark')

import os
from dotenv import load_dotenv
load_dotenv('../.env')
access = os.environ.get('AWS_ACCESS')
secret = os.environ.get('AWS_SECRET')

In [59]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql import Row
from pyspark.sql.types import ArrayType, StringType, TimestampType, DateType, StructType, DoubleType, IntegerType
from pyspark.sql.functions import *

from pyspark.ml.functions import vector_to_array
from pyspark.ml.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.feature import CountVectorizer, IDF, StopWordsRemover
from pyspark.ml import Pipeline, PipelineModel

In [32]:
conf = SparkConf() \
    .set("fs.s3a.awsAccessKeyId", access) \
    .set("fs.s3a.awsSecretAccessKey", secret) \
    .set("fs.s3a.endpoint", "s3.us-east-1.amazonaws.com") \
    .set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("fs.s3a.impl","org.apache.hadoop.fs.s3native.NativeS3FileSystem") \
    .set("com.amazonaws.services.s3.enableV4", "true")

spark = SparkSession.builder.master('local').appName('cool').config(conf=conf).getOrCreate()

In [49]:
# filename = 's3a://patellism/processed_data/2020-08-cleaned.parquet.snappy/part-00004-d2d9c5cf-46de-47d9-86df-28fefd1709e5-c000.snappy.parquet'
filename = '2020-08-cleaned-small.parquet.snappy'
df = spark.read.parquet(filename).drop('geo', 'coordinates', 'place', 'retweet_count', 'favorite_count')
df = df.withColumn('datetime', df['created_at'].cast(TimestampType())).drop('created_at')
df.show(5)

88

In [34]:
# This is a disguisting way to do this. If the data is too large could outofmemory

# Make sure the countvectorize params are the same as the below pipeline
cv = CountVectorizer(inputCol='clean_text', outputCol='output', vocabSize=5000, minDF=1)
model = cv.fit(df)
cv_df = model.transform(df)

vocab = model.vocabulary
combined = df.select(explode('clean_text').alias('col')).select(collect_list('col').alias('clean_text'))
counts = model.transform(combined).select('output').collect()

count_list = list(zip(model.vocabulary, counts[0]['output'].values))
num_removed = int(0.05*len(count_list))

to_remove = []
for i in range(num_removed):
    to_remove.append(count_list[i][0])

to_remove

['say',
 'trump',
 'make',
 'people',
 'work',
 'enough',
 'im',
 'get',
 'biden',
 'american',
 'one',
 'good',
 'need',
 'time',
 'amp',
 'president',
 'state',
 'we',
 'racist',
 'use',
 'cant',
 'democrat',
 'even',
 'think',
 'go',
 'u',
 'see',
 'right',
 'vote',
 'many',
 'america',
 'thing',
 'week',
 'read']

In [35]:
rm_freq_words = StopWordsRemover(inputCol='clean_text', outputCol='clean_rm_frequent', stopWords=to_remove)
cv_tf = CountVectorizer(inputCol='clean_rm_frequent', outputCol='features', vocabSize=5000, minDF=1)
lda_tf = LDA(k=3, maxIter=20)

tf_pipeline = Pipeline(stages=[rm_freq_words, cv_tf, lda_tf])

In [36]:
# Three groupings
#   Ungrouped
#   Time
#   Hashtags
#       Need to determine similarity metrics and method for tweets with no hashtags

In [37]:
model_tf = tf_pipeline.fit(df)

topics_tf = model_tf.stages[-1].describeTopics(maxTermsPerTopic=5)
vocabArray_tf = model_tf.stages[1].vocabulary

def covertToWord(indices):
    result = []
    for i in indices:
        result.append(vocabArray_tf[i])
    return result

udf_convertToWord = udf(covertToWord, ArrayType(StringType()))
topics_tf = topics_tf.withColumn('word', udf_convertToWord('termIndices'))
topics_tf.select('word').show(truncate=False)

+---------------------------------------+
|word                                   |
+---------------------------------------+
|[puppet, you, everyone, could, gotta]  |
|[kid, believe, country, teacher, give] |
|[faith, respect, lose, truth, election]|
+---------------------------------------+



In [38]:
transformed_tf = model_tf.transform(df)
transformed_tf.show(5)

+-------------------+--------------------+--------+--------------------+-------------------+--------------------+--------------------+--------------------+
|                 id|           full_text|hashtags|          clean_text|           datetime|   clean_rm_frequent|            features|   topicDistribution|
+-------------------+--------------------+--------+--------------------+-------------------+--------------------+--------------------+--------------------+
|1291569732789981185|@JRILLaw @Lrihend...|      []|[litigator, abili...|2020-08-06 22:59:55|[litigator, abili...|(655,[39,48,154,1...|[0.02138893474753...|
|1291569732924010496|@Jehi001 @Bibathe...|      []|[get, lose, vile,...|2020-08-06 22:59:55|[lose, vile, unca...|(655,[12,139,254,...|[0.04251318820887...|
|1291569734735917057|@granada761 @Real...|      []|[house, discrimin...|2020-08-06 22:59:55|[house, discrimin...|(655,[2,4,19,31,3...|[0.01425062821245...|
|1291569735994183685|@metawoke @Halani...|      []|[know, one, r

In [39]:
ll = model_tf.stages[-1].logLikelihood(transformed_tf)
lp = model_tf.stages[-1].logPerplexity(transformed_tf)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

The lower bound on the log likelihood of the entire corpus: -6719.647641050866
The upper bound on perplexity: 7.558658763836744


In [72]:
# Group by time
time_df = df.drop('full_text', 'hashtags')

time_df = time_df.withColumn('day_mo_yr', date_format('datetime', 'yyyy-MM-dd'))
time_df = time_df.withColumn('hour', hour('datetime'))

def amOrPm(hour):
    res = hour - 12
    if res < 0:
        return 0
    else: 
        return 1

udf_amOrPm = udf(amOrPm, IntegerType())
time_df = time_df.withColumn('am_or_pm', udf_amOrPm('hour'))

time_df = \
    time_df.withColumn('clean_text', explode('clean_text')) \
    .groupBy('day_mo_yr', 'am_or_pm') \
    .agg(collect_list('clean_text'), collect_list('id')) \
    .withColumnRenamed('collect_list(clean_text)', 'clean_text') \
    .withColumnRenamed('collect_list(id)', 'id_list')

time_df.show()

+----------+--------+--------------------+--------------------+
| day_mo_yr|am_or_pm|          clean_text|             id_list|
+----------+--------+--------------------+--------------------+
|2020-08-06|       1|[litigator, abili...|[1291569732789981...|
+----------+--------+--------------------+--------------------+



In [41]:
# cv_idf = CountVectorizer(inputCol='clean_text', outputCol='raw_features', vocabSize=5000, minDF=3)
# idf = IDF(inputCol="raw_features", outputCol="features")
# lda_tfidf = LDA(k=3, maxIter=20)

# tfidf_pipeline = Pipeline(stages=[cv_idf, idf, lda_tfidf])

In [42]:
# model_idf = tfidf_pipeline.fit(df)

# topics_idf = model_idf.stages[-1].describeTopics(maxTermsPerTopic=5)
# vocabArray_idf = model_idf.stages[0].vocabulary

# def covertToWord(indices):
#     result = []
#     for i in indices:
#         result.append(vocabArray_idf[i])
#     return result

# udf_convertToWord = udf(covertToWord, ArrayType(StringType()))
# topics_idf = topics_idf.withColumn('word', udf_convertToWord('termIndices'))
# topics_idf.select('word').show(truncate=False)

In [43]:
# transformed_idf = model_idf.transform(df)
# transformed_idf.show()

In [44]:
# ll = model_idf.stages[-1].logLikelihood(transformed_idf)
# lp = model_idf.stages[-1].logPerplexity(transformed_idf)
# print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
# print("The upper bound on perplexity: " + str(lp))